In [39]:
import os
from datetime import datetime
from dotenv import load_dotenv

#--------Google Drive Integration--------#
# from google.colab import drive, userdata
# This gives Colab access to your files in Google Drive.
# drive.mount('/content/drive')
# 'GITHUB_USERNAME' and 'GITHUB_TOKEN' saved as secrets in Colab.
# GITHUB_USERNAME = userdata.get('GITHUB_USERNAME')
# GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
# REPOSITORY_NAME = 'PyNucleus-Model' # Your repository name
# NOTEBOOK_DRIVE_PATH = "/content/drive/MyDrive/PyNucleus Project/Capstone Project.ipynb"


#--------Cursor Integration--------#
# Load environment variables from .env file
load_dotenv()

# Get GitHub credentials from environment variables
GITHUB_USERNAME = os.getenv('GITHUB_USERNAME')
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

# Print to verify the variables are loaded (remove this in production)
print(f"Username: {GITHUB_USERNAME}")
print(f"Token: {GITHUB_TOKEN[:4]}...") # Only print first 4 chars of token for security

# Repository information
REPOSITORY_NAME = 'PyNucleus-Model'
NOTEBOOK_REPO_FILENAME = "Capstone Project.ipynb"
LOG_FILENAME = "update_log.txt"

# Pull latest changes from GitHub
print("Pulling latest changes from GitHub...")
!git pull https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPOSITORY_NAME}.git main

print("Repository is up to date!")

Username: Saytor20
Token: ghp_...
Pulling latest changes from GitHub...
From https://github.com/Saytor20/PyNucleus-Model
 * branch            main       -> FETCH_HEAD
Already up to date.
Repository is up to date!


# **Data Ingestion and Preprocessing for RAG**

In [ ]:
import os
from langchain_community.document_loaders import UnstructuredFileLoader

# --- Configuration ---
# Folder where you will place all your source files (PDFs, DOCX, TXT, etc.)
INPUT_DIR = 'source_documents'

# Folder where the processed .txt files will be saved
OUTPUT_DIR = 'processed_txt_files'

# --- Main Logic ---
if __name__ == "__main__":
    # Create the input directory if it doesn't exist and give instructions
    if not os.path.exists(INPUT_DIR):
        print(f"📂 Creating directory: '{INPUT_DIR}'")
        os.makedirs(INPUT_DIR)
        print(f" Please place your files (PDF, DOCX, TXT, etc.) in the '{INPUT_DIR}' directory and run the script again.")
        exit()

    # Create the output directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    files_to_process = [f for f in os.listdir(INPUT_DIR) if os.path.isfile(os.path.join(INPUT_DIR, f))]

    if not files_to_process:
        print(f"ℹ The '{INPUT_DIR}' directory is empty. Nothing to process.")
        exit()

    print(f"--- 📄 Starting processing for {len(files_to_process)} file(s) in '{INPUT_DIR}' ---")

    for filename in files_to_process:
        input_path = os.path.join(INPUT_DIR, filename)
        
        # Create a clean output filename by changing the extension to .txt
        output_filename = os.path.splitext(os.path.basename(filename))[0] + '.txt'
        output_path = os.path.join(OUTPUT_DIR, output_filename)
        
        print(f"  ▶ Processing: {filename}")

        try:
            # The UnstructuredFileLoader automatically handles different file types.
            # The "paged" mode is faster and works for most text-based files.
            # It can be set to "elements" or "ocr" for more complex, scanned documents.
            loader = UnstructuredFileLoader(input_path, mode="paged")
            
            # The .load() method does all the work of extracting the text
            documents = loader.load()
            
            # Combine the page content into a single block of text
            full_text = "\n".join([doc.page_content for doc in documents])

            # Save the extracted text to a new .txt file
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(full_text)
                
            print(f"    • Success! Saved to: {output_path}")

        except Exception as e:
            print(f"    • Error processing {filename}: {e}")
            
    print("\n\n✅ All files processed.")

--- 📄 Starting processing for 4 file(s) in 'source_documents' ---
  ▶ Processing: 1-s2.0-S0925527302003742-main.pdf


`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


    • ✅ Success! Saved to: processed_txt_files/1-s2.0-S0925527302003742-main.txt
  ▶ Processing: Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.docx


`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
`mode='paged'` is deprecated in favor of the 'by_page' chunking strategy. Learn more about chunking here: https://docs.unstructured.io/open-source/core-functionality/chunking


    • ✅ Success! Saved to: processed_txt_files/Manuscript Draft_Can Modular Plants Lower African Industrialization Barriers.txt
  ▶ Processing: mcp_basics.txt
    • ✅ Success! Saved to: processed_txt_files/mcp_basics.txt
  ▶ Processing: feasibility_factors.txt
    • ✅ Success! Saved to: processed_txt_files/feasibility_factors.txt


✅ All files processed.


In [1]:
# FILE: scrape_web.py

import os
from googlesearch import search
from trafilatura import fetch_url, extract

# --- Configuration ---
KEYWORDS = ["modular chemical plants cost analysis 2024", "MCPs feasibility study"]
NUM_RESULTS_TO_SCRAPE = 5
OUTPUT_DIR = 'data_sources'
OUTPUT_FILENAME = "processed_web_content.txt"
FINAL_OUTPUT_PATH = os.path.join(OUTPUT_DIR, OUTPUT_FILENAME)

# --- Main Logic ---
if __name__ == "__main__":
    print(f"--- 🌐 Starting Web Scraping ---")
    query = " ".join(KEYWORDS)
    try:
        urls = list(search(query, num_results=NUM_RESULTS_TO_SCRAPE, lang="en"))
        print(f"🔍 Found {len(urls)} URLs for '{query}'.")
    except Exception as e:
        urls, print(f"❌ Search error: {e}") = [], None

    final_content = []
    for url in urls:
        print(f"  ▶ Scraping: {url}")
        raw = fetch_url(url)
        if raw and (text := extract(raw, include_comments=False)):
            final_content.append(f"--- Content from URL: {url} ---\n{text}\n\n")
            print("    • ✅ Extracted main text")
        else:
            print("    • ⚠️ No main text found or download failed")

    if final_content:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        with open(FINAL_OUTPUT_PATH, "w", encoding="utf-8") as f:
            f.write("".join(final_content))
        print(f"\n✅ Saved web content to '{FINAL_OUTPUT_PATH}'.")

SyntaxError: cannot assign to function call here. Maybe you meant '==' instead of '='? (4197401654.py, line 22)

In [43]:
# --- 4. Inspect the Results ---
import numpy as np
# --- NEW: Statistical Analysis of Chunks ---
print("\n--- Statistical Analysis & Quality Check ---")

# Calculate the lengths of all chunks
chunk_lengths = [len(chunk.page_content) for chunk in chunked_documents]

# Calculate and print key statistics
total_chunks = len(chunk_lengths)
min_size = np.min(chunk_lengths)
max_size = np.max(chunk_lengths)
avg_size = np.mean(chunk_lengths)
std_dev = np.std(chunk_lengths)

print(f"Total Chunks: {total_chunks}")
print(f"Minimum Chunk Size: {min_size} characters")
print(f"Maximum Chunk Size: {max_size} characters")
print(f"Average Chunk Size: {avg_size:.2f} characters")
print(f"Standard Deviation of Chunk Size: {std_dev:.2f}")

# --- Automated Quality Feedback ---

# 1. Check for high variation in chunk size
# A high standard deviation suggests inconsistent chunking.
if std_dev > 150:
    print(f"\n[WARNING] High chunk size variation detected (Std Dev: {std_dev:.2f}).")
    print("  > This suggests documents may have irregular structures (e.g., many short lines or lists).")
    print("  > Resulting chunks may have inconsistent levels of context.")

# 2. Check for and count potentially "orphaned" or very small chunks
small_chunk_threshold = CHUNK_SIZE * 0.20 # Chunks smaller than 20% of the target size
small_chunk_count = sum(1 for length in chunk_lengths if length < small_chunk_threshold)

if small_chunk_count > 0:
    # This check is more specific than just looking at the absolute minimum.
    print(f"\n[ADVISORY] Found {small_chunk_count} chunks smaller than {small_chunk_threshold} characters.")
    print(f"  > The smallest chunk is {min_size} characters.")
    print("  > These small chunks might lack sufficient context and could clutter search results.")
    print("  > Consider cleaning the source documents or adjusting the chunking separators.")

# Add a success message if no issues are flagged
if std_dev <= 150 and small_chunk_count == 0:
    print("\n[INFO] Chunking statistics appear healthy. Sizes are consistent.")


# --- Manual Inspection of Sample Chunks ---
# (This part remains the same)
print("\n--- Sample Chunk Preview ---")
# Print the first few chunks to get a feel for their content and structure
for i, chunk in enumerate(chunked_documents[:3]): # Print first 3 chunks
    chunk_source = os.path.basename(chunk.metadata.get('source', 'N/A'))
    print(f"\n--- Chunk {i+1} (Source: {chunk_source}, Length: {len(chunk.page_content)} chars) ---")
    print(chunk.page_content)


print("\n\nData ingestion and preprocessing is complete. The 'chunked_documents' are ready for the next stage (embedding).")


--- Statistical Analysis & Quality Check ---
Total Chunks: 4
Minimum Chunk Size: 126 characters
Maximum Chunk Size: 472 characters
Average Chunk Size: 322.00 characters
Standard Deviation of Chunk Size: 143.16

[INFO] Chunking statistics appear healthy. Sizes are consistent.

--- Sample Chunk Preview ---

--- Chunk 1 (Source: mcp_basics.txt, Length: 472 chars) ---
Modular Chemical Plants (MCPs) represent a paradigm shift in chemical process engineering. They involve constructing plants from standardized, pre-fabricated modules built off-site. This approach significantly reduces on-site construction time and costs compared to traditional stick-built plants. Key advantages include faster deployment, scalability, and potentially lower capital expenditure. However, module transportation and site integration require careful planning

--- Chunk 2 (Source: mcp_basics.txt, Length: 126 chars) ---
. MCPs are particularly suited for remote locations or projects with uncertain market demands, all

# **Data Scrapping**

In [ ]:
# scrape_web.py
import os
from googlesearch import search
from trafilatura import fetch_url, extract

# --- Configuration ---
KEYWORDS = ["modular chemical plants cost analysis 2024", "MCPs feasibility study", "pre-fabricated chemical modules logistics"]
NUM_RESULTS_TO_SCRAPE = 5
OUTPUT_DIR = 'data_sources'
WEB_OUTPUT_FILENAME = "web_scraped_content.txt"
FINAL_OUTPUT_PATH = os.path.join(OUTPUT_DIR, WEB_OUTPUT_FILENAME)

# --- Main Execution ---
if __name__ == "__main__":
    print(f"--- 🌐 Starting Web Scraping ---")
    query = " ".join(KEYWORDS)
    try:
        urls = list(search(query, num_results=NUM_RESULTS_TO_SCRAPE, lang="en"))
        print(f"🔍 Found {len(urls)} URLs for '{query}'.")
    except Exception as e:
        print(f"❌ Search error: {e}")
        urls = []

    final_content_chunks = []
    if urls:
        for url in urls:
            print(f"  ▶ Scraping: {url}")
            raw = fetch_url(url)
            if not raw:
                print("    • ❌ Download failed")
                continue
            text = extract(raw, include_comments=False, include_tables=False)
            if text:
                header = f"--- Content from URL: {url} ---\n"
                final_content_chunks.append(header + text + "\n\n")
                print("    • ✅ Extracted main text")
            else:
                print("    • ⚠️ No main text found")

    if final_content_chunks:
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        with open(FINAL_OUTPUT_PATH, "w", encoding="utf-8") as f:
            f.write("".join(final_content_chunks))
        print(f"\n✅ Success! Scraped content from {len(final_content_chunks)} URL(s) and saved to '{FINAL_OUTPUT_PATH}'.")
    else:
        print("\n❌ No content was scraped from the web.")

🔍 Found 5 URLs for 'modular chemical plants MCPs feasibility pre-fabricated chemical modules'.
  ▶ Scraping: /search?num=7
    • ❌ Download failed
  ▶ Scraping: https://www.hm-ec.com/blog-posts/modular-process-plants-hm
    • ✅ Extracted main text
  ▶ Scraping: https://www.tce.co.in/blogs/Modularization%20of%20chemical%20plants.pdf
    • ⚠️ No main text found
  ▶ Scraping: https://www.hm-ec.com/blog-posts/modular-plant-construction-hm
    • ✅ Extracted main text
  ▶ Scraping: https://dechema.de/Cost_Engineering_for_Modular_Plants/_/2024.11.14%20Cost%20Engineering%20for%20Modular%20Plants%20FINAL.pdf
    • ⚠️ No main text found

✅ Saved 2 pages to 'data_sources/web_scraped_mcp_data.txt'.
📊 Key Metrics:
   • URLs Found    : 5
   • Successful    : 2
   • Failed        : 3
   • Total Chars   : 13192
   • Total Words   : 1822


# This is the last cell of the code

In [45]:
# Simple GitHub update function
def update_github():
    !git add .
    !git commit -m "Update: Adding all files to repository"
    !git push origin main
    print("All files pushed to GitHub successfully!")

# To use it, just run:
update_github()

[main e5e9d56] Update: Adding all files to repository
 1 file changed, 46 insertions(+)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 4.48 KiB | 4.48 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/Saytor20/PyNucleus-Model.git
   db1e894..e5e9d56  main -> main
All files pushed to GitHub successfully!
